#Sistema Experto de Diagnóstico Médico

In [1]:
pip install experta

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=af54f17192669ff93e386d2c2d5055c5e88f8a1e5bac8b2b2635b5c44042b26a
  Stored in directory: /root/.cache/pip/wheels/f6/ff/aa/750fec7bf9618d87b53572def5abf3e098f853cc5ab4147656
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.65 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.


In [2]:
import collections.abc
if not hasattr(collections, 'Mapping'):
    collections.Mapping = collections.abc.Mapping

In [8]:
from experta import *

# Definimos hechos
class Sintoma(Fact):
    """Un síntoma posible"""
    pass

class Enfermedad(Fact):
    """Enfermedad con su lista de síntomas característicos"""
    pass

class Gravedad(Fact):
    """Nivel de gravedad de una enfermedad"""
    pass

class Diagnostico(Fact):
    """Diagnóstico generado por el sistema"""
    pass

class Recomendacion(Fact):
    """Recomendación asociada al diagnóstico"""
    pass


class DiagnosticoMedico(KnowledgeEngine):

    # === Diagnóstico según síntomas (con ≥ len(sintomas)/2) ===
    @Rule(Enfermedad(nombre='resfriado_comun', sintomas=MATCH.sintomas) &
          Fact(sintomas_paciente=MATCH.sintomas_paciente))
    def diagnosticar_resfriado(self, sintomas, sintomas_paciente):
        coincidencias = sum(1 for s in sintomas if s in sintomas_paciente)
        if coincidencias >= len(sintomas) / 2:
            self.declare(Diagnostico(enfermedad='resfriado_comun'))

    @Rule(Enfermedad(nombre='gripe', sintomas=MATCH.sintomas) &
          Fact(sintomas_paciente=MATCH.sintomas_paciente))
    def diagnosticar_gripe(self, sintomas, sintomas_paciente):
        coincidencias = sum(1 for s in sintomas if s in sintomas_paciente)
        if coincidencias >= len(sintomas) / 2:
            self.declare(Diagnostico(enfermedad='gripe'))

    @Rule(Enfermedad(nombre='covid_19', sintomas=MATCH.sintomas) &
          Fact(sintomas_paciente=MATCH.sintomas_paciente))
    def diagnosticar_covid(self, sintomas, sintomas_paciente):
        coincidencias = sum(1 for s in sintomas if s in sintomas_paciente)
        if coincidencias >= len(sintomas) / 2:
            self.declare(Diagnostico(enfermedad='covid_19'))

    @Rule(Enfermedad(nombre='neumonia', sintomas=MATCH.sintomas) &
          Fact(sintomas_paciente=MATCH.sintomas_paciente))
    def diagnosticar_neumonia(self, sintomas, sintomas_paciente):
        coincidencias = sum(1 for s in sintomas if s in sintomas_paciente)
        if coincidencias >= len(sintomas) / 2:
            self.declare(Diagnostico(enfermedad='neumonia'))

    # === Recomendaciones básicas según gravedad ===
    @Rule(Diagnostico(enfermedad=MATCH.enf) &
          Gravedad(enfermedad=MATCH.enf, nivel=1))
    def recomendacion_leve(self, enf):
        self.declare(Recomendacion(mensaje="Reposo en casa, hidratación y medicamentos básicos."))

    @Rule(Diagnostico(enfermedad=MATCH.enf) &
          Gravedad(enfermedad=MATCH.enf, nivel=2))
    def recomendacion_media(self, enf):
        self.declare(Recomendacion(mensaje="Consulta médica recomendada y tratamiento con supervisión."))

    @Rule(Diagnostico(enfermedad=MATCH.enf) &
          Gravedad(enfermedad=MATCH.enf, nivel=3))
    def recomendacion_grave(self, enf):
        self.declare(Recomendacion(mensaje="Atención médica urgente, posible hospitalización."))


# =========================
# Base de conocimiento
# =========================

enfermedades = [
    Enfermedad(nombre='resfriado_comun', sintomas=['tos', 'congestion_nasal', 'estornudos', 'dolor_garganta']),
    Enfermedad(nombre='gripe', sintomas=['fiebre', 'tos', 'dolor_muscular', 'fatiga', 'dolor_cabeza']),
    Enfermedad(nombre='covid_19', sintomas=['fiebre', 'tos', 'fatiga', 'dificultad_respirar', 'dolor_muscular']),
    Enfermedad(nombre='neumonia', sintomas=['tos', 'fiebre', 'dificultad_respirar', 'dolor_pecho', 'fatiga']),
]

gravedades = [
    Gravedad(enfermedad='resfriado_comun', nivel=1),
    Gravedad(enfermedad='gripe', nivel=2),
    Gravedad(enfermedad='covid_19', nivel=3),
    Gravedad(enfermedad='neumonia', nivel=3),
]

# =========================
# Ejecución
# =========================

if __name__ == '__main__':
    motor = DiagnosticoMedico()
    motor.reset()

    for enf in enfermedades:
        motor.declare(enf)

    for grav in gravedades:
        motor.declare(grav)

    sintomas_conocidos = ['fiebre', 'tos', 'dolor_cabeza', 'dolor_garganta', 'congestion_nasal','fatiga', 'dolor_muscular', 'estornudos', 'dificultad_respirar', 'dolor_pecho']

    sintomas_paciente = []
    print("🩺 Diagnóstico Médico Básico")
    print("Responde 'si' o 'no' a cada síntoma:")

    for sintoma in sintomas_conocidos:
        respuesta = input(f"¿Tienes {sintoma.replace('_', ' ')}? (si/no): ").strip().lower()
        if respuesta == 'si':
            sintomas_paciente.append(sintoma)

    motor.declare(Fact(sintomas_paciente=sintomas_paciente))

    # Ejecutamos el motor
    motor.run()

    # Mostramos resultado
    diagnostico_final = None
    recomendacion_final = None

    for hecho in motor.facts.values():
        if isinstance(hecho, Diagnostico):
            diagnostico_final = hecho['enfermedad']
        elif isinstance(hecho, Recomendacion):
            recomendacion_final = hecho['mensaje']

    if diagnostico_final:
        print(f"\n✅ Diagnóstico: {diagnostico_final}")
        print(f"💡 Recomendación: {recomendacion_final}")
    else:
        print("\n❓ No se pudo hacer un diagnóstico claro. Por favor consulta al médico.")


🩺 Diagnóstico Médico Básico
Responde 'si' o 'no' a cada síntoma:
¿Tienes fiebre? (si/no): si
¿Tienes tos? (si/no): no
¿Tienes dolor cabeza? (si/no): si
¿Tienes dolor garganta? (si/no): no
¿Tienes congestion nasal? (si/no): no
¿Tienes fatiga? (si/no): si
¿Tienes dolor muscular? (si/no): si
¿Tienes estornudos? (si/no): no
¿Tienes dificultad respirar? (si/no): si
¿Tienes dolor pecho? (si/no): no

✅ Diagnóstico: gripe
💡 Recomendación: Consulta médica recomendada y tratamiento con supervisión.
